In [29]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": [
        "2m_dewpoint_temperature",
        "2m_temperature",
        "sea_surface_temperature",
        "surface_pressure",
        "100m_u_component_of_wind",
        "10m_u_component_of_neutral_wind",
        "total_cloud_cover",
        "soil_temperature_level_1",
        "sea_ice_cover"
    ],
    "year": ["2025"],
    "month": ["01"],
    "day": ["01"],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
    "area": [90, -180, -90, 180],
    "grid": [1.0, 1.0]  # 1x1 degree resolution
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()



2025-04-22 13:00:01,277 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-22 13:00:01,278 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-22 13:00:01,654 INFO Request ID is fcfacd1d-ab75-4e0c-ad61-96d63e77f971
2025-04-22 13:00:01,729 INFO status has been updated to accepted
2025-04-22 13:00:15,427 INFO status has been updated to running
2025-04-22 13:00:23,101 INFO status has been updated to successful


'b2215eb4b21667d95d71dd47beeff0d6.grib'

In [39]:
import cfgrib
import pandas as pd
ds = cfgrib.open_datasets("b2215eb4b21667d95d71dd47beeff0d6.grib")
print(ds)

[<xarray.Dataset> Size: 6MB
Dimensions:              (time: 24, latitude: 181, longitude: 360)
Coordinates:
    number               int64 8B 0
  * time                 (time) datetime64[ns] 192B 2025-01-01 ... 2025-01-01...
    step                 timedelta64[ns] 8B 00:00:00
    depthBelowLandLayer  float64 8B 0.0
  * latitude             (latitude) float64 1kB 90.0 89.0 88.0 ... -89.0 -90.0
  * longitude            (longitude) float64 3kB -180.0 -179.0 ... 178.0 179.0
    valid_time           (time) datetime64[ns] 192B ...
Data variables:
    stl1                 (time, latitude, longitude) float32 6MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts, <xarray.Dataset> Size: 50MB
Dimensions:     (time: 24, latitude: 181, longitud

c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a

In [52]:
import xarray as xr

# Open the GRIB file
ds = cfgrib.open_datasets("b2215eb4b21667d95d71dd47beeff0d6.grib")

# Initialize a list to store DataFrames
dfs = []

# Process each dataset
for dataset in ds:
    # Convert dataset to DataFrame and reset index to include coordinates
    df = dataset.to_dataframe().reset_index()
    dfs.append(df)

# Merge DataFrames on common coordinates (time, latitude, longitude)
if len(dfs) > 1:
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.merge(df, on=['time', 'latitude', 'longitude'], how='outer')
else:
    merged_df = dfs[0]

# Select relevant columns (coordinates + variables)
variables = ['stl1', 'siconc', 'sst', 'sp', 'tcc', 't2m', 'd2m', 'u10n', 'u100', ]
output_columns = ['time', 'latitude', 'longitude'] + [col for col in merged_df.columns if col in variables]
merged_df = merged_df[output_columns]

merged_df = merged_df.reset_index()

merged_df.head()   

c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a

,index,time,latitude,longitude,stl1,siconc,sst,sp,tcc,t2m,d2m,u10n,u100
0,0,2025-01-01,-90.0,-180.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521
1,1,2025-01-01,-90.0,-179.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521
2,2,2025-01-01,-90.0,-178.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521
3,3,2025-01-01,-90.0,-177.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521
4,4,2025-01-01,-90.0,-176.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521


In [54]:
merged_df ['t2m_celsius'] = merged_df['t2m'] - 273.15
merged_df ['dewpoint_temperature_celsius'] = merged_df['d2m'] - 273.15
merged_df ['sea_surface_temperature_celsius'] = merged_df['sst'] - 273.15
merged_df ['surface_pressure_hPa'] = merged_df['sp'] * 0.01
merged_df ['surface_temperature_level_1_cel'] = merged_df['stl1'] - 273.15
merged_df ['soil_temperature_level_1']  = merged_df['stl1'] - 273.15
merged_df ['coordinates'] = merged_df['latitude'].astype(str) + ',' + merged_df['longitude'].astype(str)
merged_df.head()

,index,time,latitude,longitude,stl1,siconc,sst,sp,tcc,t2m,d2m,u10n,u100,t2m_celsius,dewpoint_temperature_celsius,sea_surface_temperature_celsius,surface_pressure_hPa,surface_temperature_level_1_cel,soil_temperature_level_1,coordinates
0,0,2025-01-01,-90.0,-180.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521,-28.085526,-32.301086,NaN,687.249512,-40.400528,-40.400528,"-90.0,-180.0"
1,1,2025-01-01,-90.0,-179.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521,-28.085526,-32.301086,NaN,687.249512,-40.400528,-40.400528,"-90.0,-179.0"
2,2,2025-01-01,-90.0,-178.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521,-28.085526,-32.301086,NaN,687.249512,-40.400528,-40.400528,"-90.0,-178.0"
3,3,2025-01-01,-90.0,-177.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521,-28.085526,-32.301086,NaN,687.249512,-40.400528,-40.400528,"-90.0,-177.0"
4,4,2025-01-01,-90.0,-176.0,232.749466,NaN,NaN,68724.953125,0.0,245.064468,240.848907,0.812546,2.246521,-28.085526,-32.301086,NaN,687.249512,-40.400528,-40.400528,"-90.0,-176.0"


In [57]:
grouped = merged_df.groupby('coordinates')
summary_df = grouped.agg({
    't2m_celsius': ['mean', 'min', 'max', 'std', 'median'],
    'dewpoint_temperature_celsius': ['mean', 'min', 'max', 'std', 'median'],
    'sea_surface_temperature_celsius': ['mean', 'min', 'max', 'std', 'median'],
    'surface_pressure_hPa': ['mean', 'min', 'max', 'std', 'median'],
    'surface_temperature_level_1_cel': ['mean', 'min', 'max', 'std', 'median'],
    'soil_temperature_level_1': ['mean', 'min', 'max', 'std', 'median'],
    'u100': ['mean', 'min', 'max', 'std', 'median'],
    'u10n': ['mean', 'min', 'max', 'std', 'median'],
    'tcc': ['mean', 'min', 'max', 'std', 'median'],
    'siconc': ['mean', 'min', 'max', 'std', 'median']
}).reset_index()

summary_df.columns = ['_'.join(col).rstrip('_') for col in summary_df.columns.values]
summary_df ['Data'] = df['time'].iloc[0]
summary_df.to_csv('era5_summary_by_coordinate.csv', index=False)

In [59]:
import cdsapi
import cfgrib
import pandas as pd
import xarray as xr


# create list of all the ranges we want to download
years = [year for year in range(1940, 2024)]
months = [month for month in range(1, 13)]
print(years)
print(months)

[1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
